In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import h5py
import tqdm
import os


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

import sys
sys.path.append('/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation')
import anatomy as anat

In [ ]:
CALC = False

In [ ]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    '''
    https://stackoverflow.com/a/18926541
    '''
    if isinstance(cmap, str):
        cmap = plt.get_cmap(cmap)
    new_cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

cmap_base = 'magma_r'
magma_r = truncate_colormap(cmap_base, 0, 0.85)

In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10

BUILD_ARRAYS = False
SAVEFIG = False
TOL=0.01
MAXBADS=5

NVARS = 3
# NVARS = 6

NPSRS = 40
NSKIES = 25
# NSKIES = 15
RED_GAMMA = None
RED2WHITE = None



In [ ]:
def get_var_data(
        target, var=int(NVARS/2), nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz'     
):

    load_data_from_file = path+f'/{target}_v{nvars}_r{nreals}_s{nskies}_shape{str(shape)}.npz' 
    load_dets_from_file = path+f'/{target}_v{nvars}_r{nreals}_s{nskies}_shape{str(shape)}_ds' 
    if red_gamma is not None and red2white is not None:
        load_dets_from_file = load_dets_from_file+f'_r2w{red2white:.1f}_rg{red_gamma:.1f}'
    load_dets_from_file = load_dets_from_file+'.npz'

    if os.path.exists(load_data_from_file) is False:
        err = f"load data file '{load_data_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    if os.path.exists(load_dets_from_file) is False:
        err = f"load dets file '{load_dets_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    file = np.load(load_data_from_file, allow_pickle=True)
    data = file['data'][var]
    params = file['params'][var]
    file.close()

    file = np.load(load_dets_from_file, allow_pickle=True)
    dsdat = file['dsdat'][var]
    file.close()

    return data, params, dsdat

In [ ]:
data, params, dsdat = get_var_data('gsmf_phi0', var=1)

In [ ]:
print(dsdat['gamma_ssi'].shape)
print(data['sspar'].shape)

In [ ]:
ssmtt = data['sspar'][0]/MSOL
ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
ssmtt = np.swapaxes(ssmtt, -2, -1)

dpssi = dsdat['gamma_ssi']

Get edges

In [ ]:
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot/MSOL
# ff_edges = data['fobs_edges']*YR

dpmin = np.min(dpssi)
dpmax = np.max(dpssi)
print(dpmin, dpmax)
dp_edges = np.geomspace(2.5e-6,1.0, NBINS)

In [ ]:
hist, mme, ffe = np.histogram2d(
    dpssi.flatten(), ssmtt.flatten(), bins=(dp_edges, mt_edges))
mtgrid, dpgrid, = np.meshgrid(mt_edges, dp_edges)

In [ ]:
snssi = dsdat['snr_ss']
snmin = np.min(snssi)
snmax = np.max(snssi)
print(snmin, snmax)
sn_edges = np.geomspace(2.e-6,25, NBINS)

In [ ]:
TARGET = 'gsmf_phi0'
NSKIES = 25

# get edges
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot[15:-10]/MSOL
sn_edges = np.geomspace(2.e-6,25, NBINS)

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

xlabel='Mass [M$_\odot$]'
ylabel='SNR'

fig, axs = plot.figax_single(
    ncols=3, sharex=True, sharey=True, height=3.5)


for ii in [0,1,2]: # vars to use
    data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES)
    ssmtt = data['sspar'][0]/MSOL
    ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
    ssmtt = np.swapaxes(ssmtt, -2, -1)

    snssi = dsdat['snr_ss']

    # get histogram
    hist, mte, sne = np.histogram2d(
        snssi.flatten(), ssmtt.flatten(), bins=(sn_edges, mt_edges))

    im = axs[ii].pcolormesh(mtgrid, sngrid, np.log10(hist), cmap=cm.BuPu)
    title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[ii].text(0, 1, title, horizontalalignment='left', verticalalignment='top', 
                 transform=axs[ii].transAxes, color='white')

# fig.colorbar(im, cax=axs[-1], label='$\log N$', orientation='vertical')
axs[0].set_ylabel(ylabel)
axs[1].set_xlabel(xlabel)
fig.tight_layout()
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.4, 
             orientation='horizontal', pad=0.22, label='$\log N$', )


In [ ]:
TARGET = 'gsmf_phi0'
NSKIES = 25

# get edges
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot[25:-12]/MSOL
sn_edges = np.geomspace(2.e-6,25, NBINS)

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

xlabel='Mass [M$_\odot$]'
ylabel='SNR'

fig, axs = plot.figax_single(
    nrows=3, sharex=True, sharey=True, height=11)

for ii in [0,1,2]: # vars to use
    data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES)
    ssmtt = data['sspar'][0]/MSOL
    ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
    ssmtt = np.swapaxes(ssmtt, -2, -1)

    snssi = dsdat['snr_ss']

    # get histogram
    hist, mte, sne = np.histogram2d(
        snssi.flatten(), ssmtt.flatten(), bins=(sn_edges, mt_edges))

    im = axs[ii].pcolormesh(mtgrid, sngrid, np.log10(hist))
    title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[ii].text(0, 1, title, horizontalalignment='left', verticalalignment='top', 
                 transform=axs[ii].transAxes, color='white', fontsize=18)
    axs[ii].set_facecolor('k')

axs[1].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='horizontal', pad=0.08, label='$\log N$', )


In [ ]:
TARGET = 'gsmf_phi0'

# get edges
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot[25:-15]/MSOL
sn_edges = np.geomspace(2.e-6,25, NBINS)

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# titles
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# colormaps
# cmaps = [cm.Greens, cm.Blues, cm.Purples]
cmaps = [cm.Oranges, cm.Greys, cm.Blues]


fig, ax = plot.figax_single()

for ii in [0,1,2]: # vars to use
    data, params, dsdat = get_var_data(target=TARGET, var=ii)
    ssmtt = data['sspar'][0]/MSOL
    ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
    ssmtt = np.swapaxes(ssmtt, -2, -1)

    snssi = dsdat['snr_ss']

    # get histogram
    hist, mte, sne = np.histogram2d(
        snssi.flatten(), ssmtt.flatten(), bins=(sn_edges, mt_edges))

    im = ax.pcolormesh(mtgrid, sngrid, np.log10(hist), cmap=cmaps[ii], alpha=0.5)
    title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    # axs[ii].text(0, 1, title, horizontalalignment='left', verticalalignment='top', 
    #              transform=axs[ii].transAxes, color='white')

ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)

fig.tight_layout()

In [ ]:
TARGET = 'hard_time'
NSKIES = 50


# get edges
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot[20:-12]/MSOL
sn_edges = np.geomspace(2.e-6,515, NBINS)

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

xlabel='Mass [M$_\odot$]'
ylabel='SNR'

fig, axs = plot.figax_single(
    nrows=3, sharex=True, sharey=True, height=11)

for ii in [0,1,2]: # vars to use
    data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES)
    ssmtt = data['sspar'][0]/MSOL
    ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
    ssmtt = np.swapaxes(ssmtt, -2, -1)

    snssi = dsdat['snr_ss']

    # get histogram
    hist, mte, sne = np.histogram2d(
        snssi.flatten(), ssmtt.flatten(), bins=(sn_edges, mt_edges))

    im = axs[ii].pcolormesh(mtgrid, sngrid, np.log10(hist), cmap=cm.magma)
    title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[ii].text(0, 1, title, horizontalalignment='left', verticalalignment='top', 
                 transform=axs[ii].transAxes, color='white', fontsize=18)
    # axs[ii].set_facecolor('k')

axs[1].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='horizontal', pad=0.08, label='$\log N$', )


In [ ]:
TARGET = 'hard_time'
NSKIES = 50

# get edges
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot[25:-15]/MSOL
sn_edges = np.geomspace(2.e-6,25, NBINS)

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# titles
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# colormaps
# cmaps = [cm.Greens, cm.Blues, cm.Purples]
cmaps = [cm.Oranges, cm.Greys, cm.Blues]


fig, ax = plot.figax_single()

for ii in [0,1,2]: # vars to use
    data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES)
    ssmtt = data['sspar'][0]/MSOL
    ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
    ssmtt = np.swapaxes(ssmtt, -2, -1)

    snssi = dsdat['snr_ss']

    # get histogram
    hist, mte, sne = np.histogram2d(
        snssi.flatten(), ssmtt.flatten(), bins=(sn_edges, mt_edges))

    im = ax.pcolormesh(mtgrid, sngrid, np.log10(hist), cmap=cmaps[ii], alpha=0.5)
    title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    # axs[ii].text(0, 1, title, horizontalalignment='left', verticalalignment='top', 
    #              transform=axs[ii].transAxes, color='white')

ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)

fig.tight_layout()

# Hard_time 5ax

In [ ]:
TARGET = 'hard_time'
NSKIES = 100
NVARS = 21

if CALC:
    # get edges
    NBINS = 40
    sam = holo.sams.Semi_Analytic_Model()
    mt_edges = sam.mtot[20:-12]/MSOL
    sn_edges = np.geomspace(2.e-6,515, NBINS)

    # get histograms
    ssmtt = []
    snssi = []
    hist = []
    text = []
    for ii in [0,5,10,15,20]: # vars to use
        data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES, nvars=NVARS,
                        path = '/Users/emigardiner/GWs/holodeck/output/anatomy_09B'  )
        _ssmtt = data['sspar'][0]/MSOL
        _ssmtt = np.repeat(_ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
        _ssmtt = np.swapaxes(_ssmtt, -2, -1)
        ssmtt.append(_ssmtt)

        _snssi = dsdat['snr_ss']
        snssi.append(_snssi)
        # get histogram
        _hist, mte, sne = np.histogram2d(
            _snssi.flatten(), _ssmtt.flatten(), bins=(sn_edges, mt_edges))
        hist.append(_hist)

        _text = '%.2f' % (params[TARGET])
        text.append(_text)
    np.savez('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
            +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz',
            mt_edges=mt_edges, sn_edges=sn_edges,
            ssmtt = ssmtt, snssi=snssi, hist=hist, text=text)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
            +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    ssmtt = file['ssmtt']
    snssi = file['snssi']
    hist = file['hist']
    text = file['text']
    file.close()

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0.01, 0.99, '$\\tau_\mathrm{hard}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.03, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_single(
    nrows=5, sharex=True, sharey=True, height=16)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist[aa]), cmap=cm.magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0, 1, '$\\tau_\mathrm{hard}$=%s' % text[aa], 
                 horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=18)
    # axs[ii].set_facecolor('k')

# set labels
axs[2].set_ylabel(ylabel)
axs[-1].set_xlabel(xlabel)
fig.tight_layout()

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='horizontal', pad=0.08, label='$\log N$', )


# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)

# Hard_gamma_inner 5ax

In [ ]:
TARGET = 'hard_gamma_inner'
NSKIES = 100
NVARS = 21

if CALC:
    # get edges
    NBINS = 40
    sam = holo.sams.Semi_Analytic_Model()
    mt_edges = sam.mtot[20:-12]/MSOL
    sn_edges = np.geomspace(2.e-6,515, NBINS)

    # get histograms
    ssmtt = []
    snssi = []
    hist = []
    text = []
    for ii in [0,5,10,15,20]: # vars to use
        data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES, nvars=NVARS,
                        path = '/Users/emigardiner/GWs/holodeck/output/anatomy_09B'  )
        _ssmtt = data['sspar'][0]/MSOL
        _ssmtt = np.repeat(_ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
        _ssmtt = np.swapaxes(_ssmtt, -2, -1)
        ssmtt.append(_ssmtt)

        _snssi = dsdat['snr_ss']
        snssi.append(_snssi)
        # get histogram
        _hist, mte, sne = np.histogram2d(
            _snssi.flatten(), _ssmtt.flatten(), bins=(sn_edges, mt_edges))
        hist.append(_hist)

        _text = '%.2f' % (params[TARGET])
        text.append(_text)
    np.savez('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
            +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz',
            mt_edges=mt_edges, sn_edges=sn_edges,
            ssmtt = ssmtt, snssi=snssi, hist=hist, text=text)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
            +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    ssmtt = file['ssmtt']
    snssi = file['snssi']
    hist = file['hist']
    text = file['text']
    file.close()
    

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0.01, 0.99, '$\\nu_\mathrm{inner}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.03, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)

SNR or DP vs. Frequency

# GSMF_phi0 5ax

In [ ]:
TARGET = 'gsmf_phi0'
NSKIES = 100
NVARS = 21

if CALC:
    # get edges
    NBINS = 40
    sam = holo.sams.Semi_Analytic_Model()
    mt_edges = sam.mtot[20:-10]/MSOL
    sn_edges = np.geomspace(2.e-6,515, NBINS)

    # get histograms
    ssmtt = []
    snssi = []
    hist = []
    text = []
    for ii in [0,5,10,15,20]: # vars to use
        data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES, nvars=NVARS,
                        path = '/Users/emigardiner/GWs/holodeck/output/anatomy_09B'  )
        _ssmtt = data['sspar'][0]/MSOL
        _ssmtt = np.repeat(_ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
        _ssmtt = np.swapaxes(_ssmtt, -2, -1)
        ssmtt.append(_ssmtt)

        _snssi = dsdat['snr_ss']
        snssi.append(_snssi)
        # get histogram
        _hist, mte, sne = np.histogram2d(
            _snssi.flatten(), _ssmtt.flatten(), bins=(sn_edges, mt_edges))
        hist.append(_hist)

        _text = '%.2f' % (params[TARGET])
        text.append(_text)
    np.savez('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
            +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz',
            mt_edges=mt_edges, sn_edges=sn_edges,
            ssmtt = ssmtt, snssi=snssi, hist=hist, text=text)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
            +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    ssmtt = file['ssmtt']
    snssi = file['snssi']
    hist = file['hist']
    text = file['text']
    file.close()

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0.01, 0.99, '$\psi_0$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.03, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)

# Gsmf_mchar0_log10 5ax

In [ ]:
TARGET = 'gsmf_mchar0_log10'
NSKIES = 100
NVARS = 21

if CALC:
    # get edges
    NBINS = 40
    sam = holo.sams.Semi_Analytic_Model()
    mt_edges = sam.mtot[20:-6]/MSOL
    sn_edges = np.geomspace(2.e-6,515, NBINS)

    # get histograms
    ssmtt = []
    snssi = []
    hist = []
    text = []
    for ii in [0,5,10,15,20]: # vars to use
        data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES, nvars=NVARS,
                        path = '/Users/emigardiner/GWs/holodeck/output/anatomy_09B'  )
        _ssmtt = data['sspar'][0]/MSOL
        _ssmtt = np.repeat(_ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
        _ssmtt = np.swapaxes(_ssmtt, -2, -1)
        ssmtt.append(_ssmtt)

        _snssi = dsdat['snr_ss']
        snssi.append(_snssi)
        # get histogram
        _hist, mte, sne = np.histogram2d(
            _snssi.flatten(), _ssmtt.flatten(), bins=(sn_edges, mt_edges))
        hist.append(_hist)

        _text = '%.2f' % (params[TARGET])
        text.append(_text)
    np.savez('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
            +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz',
            mt_edges=mt_edges, sn_edges=sn_edges,
            ssmtt = ssmtt, snssi=snssi, hist=hist, text=text)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
            +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    ssmtt = file['ssmtt']
    snssi = file['snssi']
    hist = file['hist']
    text = file['text']
    file.close()

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0.01, 0.99, '$M_\mathrm{char,0}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.03, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)

# mmb_mamp_log10 5ax

In [ ]:
TARGET = 'mmb_mamp_log10'
NSKIES = 100
NVARS = 21

if CALC:
    # get edges
    NBINS = 40
    sam = holo.sams.Semi_Analytic_Model()
    mt_edges = sam.mtot[20:-8]/MSOL
    sn_edges = np.geomspace(2.e-6,515, NBINS)

    # get histograms
    ssmtt = []
    snssi = []
    hist = []
    text = []
    for ii in [0,5,10,15,20]: # vars to use
        data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES, nvars=NVARS,
                        path = '/Users/emigardiner/GWs/holodeck/output/anatomy_09B'  )
        _ssmtt = data['sspar'][0]/MSOL
        _ssmtt = np.repeat(_ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
        _ssmtt = np.swapaxes(_ssmtt, -2, -1)
        ssmtt.append(_ssmtt)

        _snssi = dsdat['snr_ss']
        snssi.append(_snssi)
        # get histogram
        _hist, mte, sne = np.histogram2d(
            _snssi.flatten(), _ssmtt.flatten(), bins=(sn_edges, mt_edges))
        hist.append(_hist)

        _text = '%.2f' % (params[TARGET])
        text.append(_text)
    np.savez('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
    +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz',
    mt_edges=mt_edges, sn_edges=sn_edges,
    ssmtt = ssmtt, snssi=snssi, hist=hist, text=text)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
            +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    ssmtt = file['ssmtt']
    snssi = file['snssi']
    hist = file['hist']
    text = file['text']
    file.close()

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0.01, 0.99, '$\mu$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.03, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)

# mmb_scatter_dex

In [ ]:
TARGET = 'mmb_scatter_dex'
NSKIES = 100
NVARS = 21

if CALC:
    # get edges
    NBINS = 40
    sam = holo.sams.Semi_Analytic_Model()
    mt_edges = sam.mtot[20:-8]/MSOL
    sn_edges = np.geomspace(2.e-6,515, NBINS)

    # get histograms
    ssmtt = []
    snssi = []
    hist = []
    text = []
    for ii in [0,5,10,15,20]: # vars to use
        data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES, nvars=NVARS,
                        path = '/Users/emigardiner/GWs/holodeck/output/anatomy_09B'  )
        _ssmtt = data['sspar'][0]/MSOL
        _ssmtt = np.repeat(_ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
        _ssmtt = np.swapaxes(_ssmtt, -2, -1)
        ssmtt.append(_ssmtt)

        _snssi = dsdat['snr_ss']
        snssi.append(_snssi)
        # get histogram
        _hist, mte, sne = np.histogram2d(
            _snssi.flatten(), _ssmtt.flatten(), bins=(sn_edges, mt_edges))
        hist.append(_hist)

        _text = '%.2f' % (params[TARGET])
        text.append(_text)
        
    np.savez('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
            +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz',
            mt_edges=mt_edges, sn_edges=sn_edges,
            ssmtt = ssmtt, snssi=snssi, hist=hist, text=text)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_09B/figdata'
            +f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    ssmtt = file['ssmtt']
    snssi = file['snssi']
    hist = file['hist']
    text = file['text']
    file.close()

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0.01, 0.99, '$\epsilon_\mu$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.03, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)